In [1]:
import os, ssl
import requests
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup as bs
import time
from time import sleep
import tabula
from tabula import read_pdf
import numpy
import re
from sqlalchemy import create_engine

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## Cases per City in OC  

In [3]:
url = "https://ochca.maps.arcgis.com/apps/opsdashboard/index.html#/2a169f85c2254dd7b43f95b095208356"
browser.visit(url)
time.sleep(30)

In [4]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("nav", class_="feature-list")

In [5]:
confirmed_case_data = data[0].find_all("p")
confirmed_case_city = []
for i in confirmed_case_data:
    a = i.text
    b = a.split("-")
    city_name = b[0]
    confirmed_case = b[1].split(" ")[0][1:]
    confirmed_case_city.append([city_name,confirmed_case]) 

In [6]:
death_case_data = data[1].find_all("p")
death_case_city = []
for i in death_case_data:
    a = i.text
    b = a.split("-")
    city_name = b[0]
    if("Less"in b[1]):
        death_case = b[1][:-12]
    else:
        death_case = b[1].split(" ")[1]
    death_case_city.append([city_name, death_case])  

In [7]:
oc_case_df = pd.DataFrame(confirmed_case_city)
oc_death_df = pd.DataFrame(death_case_city)

In [8]:
oc_case_df.columns = ["City","Confirmed_case"]
oc_death_df.columns = ["City","Death_case"]

In [9]:
orange_df = oc_case_df.merge(oc_death_df,how="inner", on="City")

In [10]:
orange_df["County"]=""
for index,row in orange_df.iterrows():
    orange_df.loc[index,"County"] = "Orange"

In [11]:
orange_county_df = orange_df[["County","City","Confirmed_case","Death_case"]]

In [12]:
orange_county_df.head(10)

,County,City,Confirmed_case,Death_case
0,Orange,Aliso Viejo,231,1
1,Orange,Anaheim,5047,123
2,Orange,Brea,264,5
3,Orange,Buena Park,808,7
4,Orange,Costa Mesa,931,6
5,Orange,Coto de Caza,17,Less than 5
6,Orange,Cypress,281,6
7,Orange,Dana Point,153,0
8,Orange,Fountain Valley,296,8
9,Orange,Fullerton,1364,23


## Cases per City in LA County

In [13]:
url = "http://publichealth.lacounty.gov/media/Coronavirus/locations.htm"
browser.visit(url)

In [14]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("tr")

In [15]:
LA_case = []
for row in data:
    list_td = row.find_all("td")
    temp_list = []
    for item in list_td:
        if(item.text):
            temp_list.append(item.text)
    if(temp_list):
        if "City of " in temp_list[0]:
            city_data = temp_list[0][8:];
            temp_list[0] = city_data;
            LA_case.append(temp_list)

In [16]:
la_case_df = pd.DataFrame(LA_case)
la_case_df.columns= ["City", "Confirmed_case", "Case_Rate", "Death_case", "Death_Rate"]

In [17]:
la_case_df["County"]=""

In [18]:
for index,row in la_case_df.iterrows():
    la_case_df.loc[index,"County"] = "Los Angeles"

In [19]:
la_county_df = la_case_df[["County","City","Confirmed_case","Death_case"]]
la_county_df.head(10)

,County,City,Confirmed_case,Death_case
0,Los Angeles,Agoura Hills,89,1
1,Los Angeles,Alhambra,699,14
2,Los Angeles,Arcadia,259,12
3,Los Angeles,Artesia,181,3
4,Los Angeles,Avalon,5,0
5,Los Angeles,Azusa,784,9
6,Los Angeles,Baldwin Park,1561,40
7,Los Angeles,Bell,905,26
8,Los Angeles,Bell Gardens,1088,13
9,Los Angeles,Bellflower,1374,39


## Cases per City in San Diego County

In [20]:
# url = "https://www.sandiegocounty.gov/content/dam/sdc/hhsa/programs/phs/Epidemiology/COVID-19%20Daily%20Update_City%20of%20Residence.pdf"

In [21]:
# df = read_pdf(url)

In [22]:
# sandiego_df = df[0].drop([0])

In [23]:
# sandiego_df.head()

In [24]:
# for index,row in sandiego_df.iterrows():
#     split_data = row[1].split()
#     length = len(split_data)
#     count = split_data[0]
#     if(length==3):
#         percentage_total = split_data[1]
#         rate_per_100000 = split_data[2]
#     elif (length==2):
#         percentage_total = split_data[1]
#         rate_per_100000 = ""
#     elif(length==1):
#         percentage_total = ""
#         rate_per_100000 = ""
#     sandiego_df.loc[index,"count"] = count
#     sandiego_df.loc[index,"percentage_total"] = percentage_total
#     sandiego_df.loc[index,"rate_per_100000"] = rate_per_100000

In [25]:
# sandiego_df.columns = ["City", "Summary", "Confirmed_case", "Total_Rate", "Rate_per_10000"]

In [26]:
# sandiego_df["County"]=""
# sandiego_df["Death_case"]=""

In [27]:
# for index,row in sandiego_df.iterrows():
#     sandiego_df.loc[index,"County"] = "San Diego"

In [28]:
# sandiego_county_df = sandiego_df[["County","City","Confirmed_case","Death_case"]]
# sandiego_county_df.head()

## Cases per City in Santa Barbara County

In [29]:
url = "https://publichealthsbc.org/status-reports/"
browser.visit(url)

In [30]:
html = browser.html
soup = bs(html,'html.parser')
data = soup.find_all("table")

In [31]:
a = data[0].find_all("tr")

In [32]:
sb_list = []
for i in a:
    j = i.find_all("td")
    temp_data = []
    for data in j:
        temp_data.append(data.text)
    sb_list.append(temp_data)

In [33]:
sb_df = pd.DataFrame(sb_list)

In [34]:
sb_df["County"]=""
sb_df.head(10)

,0,1,2,3,4,5,County
0,Geographic Area as of 7/17/2020,DailyCases,Total Confirmed Cases,Recovered by Region,Active Cases by Region,Number of Deaths,
1,SOUTH COUNTY UNINCORPORATED AREA includes comm...,3,88,81,6,0,
2,CITY OF SANTA BARBARA and the unincorporated a...,19,562,509,51,3,
3,CITY OF GOLETA,4,101,89,10,2,
4,COMMUNITY OF ISLA VISTA,0,29,28,1,0,
5,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,3,77,71,5,1,
6,SANTA YNEZ VALLEY including the Cities of Solv...,2,51,41,10,0,
7,CITY OF LOMPOC and the communities of Mission ...,18,288,241,43,4,
8,FEDERAL PRISON IN LOMPOC,6,1007,997,7,3,
9,CITY OF SANTA MARIA,69,2099,1862,220,18,


In [35]:
sb_df.columns = ["City", "DailyCases", "Confirmed_case","recovered","active","Death_case","County"]

In [36]:
sb_df.drop([0],inplace=True)

In [37]:
for index,row in sb_df.iterrows():
    sb_df.loc[index,"County"] = "Santa Barbara"

In [38]:
santabarbara_county_df=sb_df[["County","City","Confirmed_case","Death_case"]]
santabarbara_county_df.head(10)

,County,City,Confirmed_case,Death_case
1,Santa Barbara,SOUTH COUNTY UNINCORPORATED AREA includes comm...,88,0
2,Santa Barbara,CITY OF SANTA BARBARA and the unincorporated a...,562,3
3,Santa Barbara,CITY OF GOLETA,101,2
4,Santa Barbara,COMMUNITY OF ISLA VISTA,29,0
5,Santa Barbara,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,77,1
6,Santa Barbara,SANTA YNEZ VALLEY including the Cities of Solv...,51,0
7,Santa Barbara,CITY OF LOMPOC and the communities of Mission ...,288,4
8,Santa Barbara,FEDERAL PRISON IN LOMPOC,1007,3
9,Santa Barbara,CITY OF SANTA MARIA,2099,18
10,Santa Barbara,COMMUNITY OF ORCUTT,143,0


## Cases per City in Riverside County

In [39]:
url = "https://www.rivcoph.org/Portals/0/Documents/CoronaVirus/July/Dashboard/4/City_Report%20071720.pdf?ver=2020-07-17-152309-490&timestamp=1595024652273"
df = read_pdf(url, pages=1)
df2 = read_pdf(url, pages=2)

In [40]:
riverside_df = df[0]
riverside2_df = df2[1]
riverside_df["County"]=""

In [41]:
for index,row in riverside_df.iterrows():
    riverside_df.loc[index,"County"] = "Riverside"
    a1 = row['Current Cases']
    a2 = row['New Cases']
    a3 = row['Current Deaths']
    a4 = row['New Deaths']
    if(','in str(a1)):
        b1 = str(a1).split(",")
        a1 = b1[0]+b1[1]
    if(','in str(a2)):
        b2 = str(a2).split(",")
        a2 = b2[0]+b2[1]
    if(','in str(a3)):
        b3 = str(a3).split(",")
        a3 = b3[0]+b3[1]
    if(','in str(a4)):
        b4 = str(a4).split(",")
        a4 = b4[0]+b4[1]
    riverside_df.loc[index,"Confirmed_case"] = str(int(a1)+int(a2))
    riverside_df.loc[index,"Death_case"] = str(int(a3)+int(a4))    

In [42]:
riverside_df.rename(columns={"Name":"City"},inplace=True)

In [43]:
riverside_county_df=riverside_df[["County","City","Confirmed_case","Death_case"]]
riverside_county_df.head(10)

,County,City,Confirmed_case,Death_case
0,Riverside,County Jails,274,2
1,Riverside,State Prison,1188,1
2,Riverside,Incorporated City,23828,528
3,Riverside,Banning,328,10
4,Riverside,Beaumont,527,12
5,Riverside,Blythe,272,3
6,Riverside,Calimesa,73,1
7,Riverside,Canyon Lake,39,0
8,Riverside,Cathedral City,1100,15
9,Riverside,Coachella,1601,15


In [44]:
riverside2_df.drop([0,1,2,37,38,39,40],inplace=True)

In [45]:
riverside2_df.rename(columns={"NameCurrent CasesNew CasesCurrent DeathsNew DeathsPercentage ** of Overall Total":"data"},inplace=True)

In [46]:
for index,row in riverside2_df.iterrows():
    temp_data = row["data"][:-5]
    a = re.split("(\d+)",temp_data)
    riverside2_df.loc[index,"City"] = a[0]
    riverside2_df.loc[index,"Confirmed_case"] = str(int(a[1][-3])+int(a[1][:-3]))
    riverside2_df.loc[index,"Death_case"] = str(int(a[1][-1])+int(a[1][-2]))
    riverside2_df.loc[index,"County"]="Riverside"
    

In [47]:
riverside_county_df=pd.concat([riverside_county_df, riverside2_df[["County","City","Confirmed_case","Death_case"]]])
riverside_county_df.head(10)

,County,City,Confirmed_case,Death_case
0,Riverside,County Jails,274,2
1,Riverside,State Prison,1188,1
2,Riverside,Incorporated City,23828,528
3,Riverside,Banning,328,10
4,Riverside,Beaumont,527,12
5,Riverside,Blythe,272,3
6,Riverside,Calimesa,73,1
7,Riverside,Canyon Lake,39,0
8,Riverside,Cathedral City,1100,15
9,Riverside,Coachella,1601,15


## Cases per City in San Bernadino County

In [48]:
url1 = "Resources/COVID-19 PC Surveillance Dashboard_confirmed_files/saved_resource.html"
# url2 = "COVID-19 PC Surveillance Dashboard_death.html"

In [49]:
with open(url1, encoding = 'utf-8') as file:
    html = file.read()   
soup = bs(html,'html.parser')

In [50]:
data = soup.find_all("tbody")

In [51]:
a = data[0].find_all("tr")

In [52]:
list_index = len(a)-1;
san_bernadino_list = [];
b = a[0].find_all("p")
san_bernadino_list.append([b[0].text[2:], b[1].text, b[3].text, b[5].text])
for index in range(list_index):
    sb_value = a[index+1].find_all("p");
    temp_list = [];
    for j in range(len(sb_value)):
        if(j==0):
            temp_list.append(sb_value[j].text[2:])
        else:
            temp_list.append(sb_value[j].text)
    san_bernadino_list.append(temp_list)

In [53]:
san_bernadino_df = pd.DataFrame(san_bernadino_list)

In [54]:
san_bernadino_df.drop([0],inplace=True)

In [55]:
san_bernadino_df.head(10)

,0,1,2,3
1,Adelanto,259,5,264
2,Angelus Oaks,0,1,1
3,Apple Valley,318,2,320
4,Argus,0,0,0
5,Arrowbear,0,0,0
6,Baker,3,0,3
7,Baldy Mesa,3,0,3
8,Barstow,92,0,92
9,Big Bear City,14,1,15
10,Big Bear Lake,18,0,18


In [56]:
url2 = "Resources/COVID-19 PC Surveillance Dashboard_death_files/saved_resource.html"
with open(url2, encoding = 'utf-8') as file:
    html = file.read()   
soup = bs(html,'html.parser')

In [57]:
data = soup.find_all("tbody")

In [58]:
a = data[1].find_all("tr")

In [59]:
list_index = len(a)-1;
san_bernadino_list = [];
b = a[0].find_all("p")
san_bernadino_list.append([b[0].text, b[1].text])
for index in range(list_index):
    sb_value = a[index+1].find_all("p");
    temp_list = [];
    for j in range(len(sb_value)):
        if(j==0):
            temp_list.append(sb_value[j].text[0:])
        else:
            temp_list.append(sb_value[j].text)
    san_bernadino_list.append(temp_list)

In [60]:
browser.quit()

In [61]:
san_death_df = pd.DataFrame(san_bernadino_list)

In [62]:
san_death_df.drop([0],inplace=True)

In [63]:
san_death_df.head(10)

,0,1
1,Adelanto,5
2,Apple Valley,2
3,Barstow,2
4,Bloomington,5
5,Blue Jay,1
6,Chino,20
7,Chino Hills,4
8,Colton,20
9,Crestline,2
10,Fontana,24


In [64]:
san_bernadino_df.columns=["Location","Community","Outbreak","Total"]

In [65]:
san_death_df.columns=["Location","Deaths"]

In [66]:
total_san_bernadino_df = san_bernadino_df.merge(san_death_df,how='left', on = 'Location')

In [67]:
total_san_bernadino_df.fillna(0, inplace=True)

In [68]:
total_san_bernadino_df["County"]=""
total_san_bernadino_df.columns=["City","Community","Outbreak","Confirmed_case","Death_case","County"]
total_san_bernadino_df.head(10)

,City,Community,Outbreak,Confirmed_case,Death_case,County
0,Adelanto,259,5,264,5,
1,Angelus Oaks,0,1,1,0,
2,Apple Valley,318,2,320,2,
3,Argus,0,0,0,0,
4,Arrowbear,0,0,0,0,
5,Baker,3,0,3,0,
6,Baldy Mesa,3,0,3,0,
7,Barstow,92,0,92,2,
8,Big Bear City,14,1,15,0,
9,Big Bear Lake,18,0,18,0,


In [69]:
for index,row in total_san_bernadino_df.iterrows():
    total_san_bernadino_df.loc[index,"County"] = "San Bernardino"

In [70]:
sanbernadino_county_df=total_san_bernadino_df[["County","City","Confirmed_case","Death_case"]]
sanbernadino_county_df.head(10)

,County,City,Confirmed_case,Death_case
0,San Bernardino,Adelanto,264,5
1,San Bernardino,Angelus Oaks,1,0
2,San Bernardino,Apple Valley,320,2
3,San Bernardino,Argus,0,0
4,San Bernardino,Arrowbear,0,0
5,San Bernardino,Baker,3,0
6,San Bernardino,Baldy Mesa,3,0
7,San Bernardino,Barstow,92,2
8,San Bernardino,Big Bear City,15,0
9,San Bernardino,Big Bear Lake,18,0


## Total Cases for S. California (except Imperial, Ventura counties)

In [71]:
# OMITS SAN DIEGO
total_city_df = pd.concat([orange_county_df,la_county_df,riverside_county_df,sanbernadino_county_df,santabarbara_county_df])

In [72]:
total_city_df.columns=["county","city","totalcountconfirmed","totalcountdeaths"]
total_city_df.head(10)

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,231,1
1,Orange,Anaheim,5047,123
2,Orange,Brea,264,5
3,Orange,Buena Park,808,7
4,Orange,Costa Mesa,931,6
5,Orange,Coto de Caza,17,Less than 5
6,Orange,Cypress,281,6
7,Orange,Dana Point,153,0
8,Orange,Fountain Valley,296,8
9,Orange,Fullerton,1364,23


## Clean Up

In [73]:
total_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 284 entries, 0 to 14
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   county               284 non-null    object
 1   city                 284 non-null    object
 2   totalcountconfirmed  284 non-null    object
 3   totalcountdeaths     284 non-null    object
dtypes: object(4)
memory usage: 11.1+ KB


In [74]:
total_city_df.drop(total_city_df[(total_city_df['city'] == 'Grand Total') | (total_city_df['city'] == 'Incorporated City') | (total_city_df['city'] == 'Out of County') | (total_city_df['city'] == 'Pending') | (total_city_df['city'] == 'Total') | (total_city_df['city'] == 'Undetermined')].index, inplace = True)
total_city_df.head(10)

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,231,1
1,Orange,Anaheim,5047,123
3,Orange,Buena Park,808,7
4,Orange,Costa Mesa,931,6
5,Orange,Coto de Caza,17,Less than 5
6,Orange,Cypress,281,6
7,Orange,Dana Point,153,0
8,Orange,Fountain Valley,296,8
9,Orange,Fullerton,1364,23
10,Orange,Garden Grove,1524,14


In [75]:
total_city_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 257 entries, 0 to 11
Data columns (total 4 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   county               257 non-null    object
 1   city                 257 non-null    object
 2   totalcountconfirmed  257 non-null    object
 3   totalcountdeaths     257 non-null    object
dtypes: object(4)
memory usage: 10.0+ KB


In [76]:
total_city_df.loc[total_city_df['county'] == 'Riverside'].head(10)

,county,city,totalcountconfirmed,totalcountdeaths
0,Riverside,County Jails,274,2
1,Riverside,State Prison,1188,1
3,Riverside,Banning,328,10
4,Riverside,Beaumont,527,12
5,Riverside,Blythe,272,3
6,Riverside,Calimesa,73,1
7,Riverside,Canyon Lake,39,0
8,Riverside,Cathedral City,1100,15
9,Riverside,Coachella,1601,15
10,Riverside,Corona,1491,20


In [77]:
total_city_df['city'].replace({'PiÃ±on Hills': 'Pinion Hills',
                               'TN PalmsÂ Marine Base': 'Twentynine Palms Marine Base',
                               'Chino*': 'Chino',
                               'CITY OF GOLETA': 'Goleta',
                               'CITY OF LOMPOC and the communities of Mission Hills and Vandenberg Village': 'Lompoc (w Mission Hills and Vandenberg Village)',
                               'CITY OF SANTA BARBARA and the unincorporated area of Mission Canyon': 'Santa Barbara (w Unincorporated Mission Canyon)',
                               'CITY OF SANTA MARIA': 'Santa Maria',
                               'Commerce*': 'Commerce',
                               'COMMUNITY OF ISLA VISTA': 'Isla Vista (community)',
                               'COMMUNITY OF ORCUTT': ' Orcutt (community)',
                               'FEDERAL PRISON IN LOMPOC': 'Federal Prison in Lompoc',
                               'Lancaster*': 'Lancaster',
                               'Lynwood*': 'Lynwood',
                               'San Dimas*': 'San Dimas',
                               'SANTA YNEZ VALLEY including the Cities of Solvang & Buellton, and the communities of Santa Ynez, Los Alamos, Los Olivos and Ballard': 'Santa Ynez Valley (w Solvang & Buellton, and Santa Ynez, Los Alamos, Los Olivos and Ballard communities',
                               'SOUTH COUNTY UNINCORPORATED AREA includes communities of Montecito, Summerland and the City of Carpinteria': 'South County Unincorp Areas: Montecito, Summerland communities and Carpinteria',
                               'UNINCORPORATED AREA OF THE GOLETA VALLEY AND GAVIOTA': 'Unincorporated Areas: Goleta Valley, Gaviota',
                               'UNINCORPORATED AREAS of Sisquoc, Casmalia, Garey,\xa0 Cuyama, New Cuyama, and the City of Guadalupe': 'Unincorporated Areas: Sisquoc, Casmalia, Garey, Cuyama, New Cuyama communities and Guadalupe'
                              },inplace=True)

# total_city_df.sort_values(by=['county','city'])
# total_city_df.loc[total_city_df['county'] == 'San Bernardino']
total_city_df.head(10)

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,231,1
1,Orange,Anaheim,5047,123
3,Orange,Buena Park,808,7
4,Orange,Costa Mesa,931,6
5,Orange,Coto de Caza,17,Less than 5
6,Orange,Cypress,281,6
7,Orange,Dana Point,153,0
8,Orange,Fountain Valley,296,8
9,Orange,Fullerton,1364,23
10,Orange,Garden Grove,1524,14


In [78]:
#SOLVING THE MYSTERY OF THE UNCORRECTED LINE
# if 'UNINCORPORATED AREAS of Sisquoc, Casmalia, Garey,Â  Cuyama, New Cuyama, and the City of Guadalupe' in total_city_df.city:
#     print('element exsits')
    
# total_city_df.loc[total_city_df['county'] == 'Santa Barbara']['city'][11]

In [79]:
# UPDATING "LESS THAN 5" TO "0".
total_city_df['totalcountdeaths'].replace({' Less than 5 ': '0'},inplace=True)

In [80]:
# REMOVING COMMAS FROM NUMBERS
for index,row in total_city_df.iterrows():
    a1 = row['totalcountconfirmed']
    if(','in str(a1)):
        b1 = str(a1).split(",")
        a1 = b1[0]+b1[1]
    total_city_df.loc[index,"totalcountconfirmed"] = str(int(a1))
    
total_city_df.head(10)

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,231,1
1,Orange,Anaheim,5047,123
3,Orange,Buena Park,808,7
4,Orange,Costa Mesa,931,6
5,Orange,Coto de Caza,17,0
6,Orange,Cypress,281,6
7,Orange,Dana Point,153,0
8,Orange,Fountain Valley,296,8
9,Orange,Fullerton,1364,23
10,Orange,Garden Grove,1524,14


## Load to SQL Tables

In [82]:
rds_connection_string = "postgres:postgres@localhost:5432/covid-vaccine-plz"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [83]:
engine.table_names()

['citywide_table', 'statewide_table']

In [84]:
total_city_df.to_sql(name='citywide_table',con=engine,if_exists='append',index=False)

In [85]:
pd.read_sql_query('select* from citywide_table',con=engine)

,county,city,totalcountconfirmed,totalcountdeaths
0,Orange,Aliso Viejo,231,1
1,Orange,Anaheim,5047,123
2,Orange,Buena Park,808,7
3,Orange,Costa Mesa,931,6
4,Orange,Coto de Caza,17,0
...,...,...,...,...
252,Santa Barbara,Lompoc (w Mission Hills and Vandenberg Village),153,4
253,Santa Barbara,Federal Prison in Lompoc,296,3
254,Santa Barbara,Santa Maria,1364,18
255,Santa Barbara,Orcutt (community),1524,0


In [86]:
#EXPORTING TO .CSV FILE
total_city_df.to_csv("./CSV/citywide.csv", index=False)